# Neeps - Easy

The "Neeps" database includes details of all teaching events in the School of Computing at Napier University in Semester 1 of the 2000/2001 academic year.

Graduated questions

The timetable database contains the following tables:

- ut_staff

id | name
---|-----
co.ACg | Cumming, Andrew
co.ACr | Crerar, Dr Alison
co.AFA | Armitage, Dr Alistair
 | ... Result truncated.

The staff table simply gives the unique identifier for each member of staff together with their full name.

- ut_student

id | name | sze | parent
---|------|----:|----------
co1.BAe | BAe | 15 | 
co1.CO | Computing 1st Year | 150 | 
co1.CO.a | Computing 1st Year a | 20 | co1.CO
co1.CO.c | Computing 1st Year c | 20 | co1.CO
co1.CO.d | Computing 1st Year d | 20 | co1.CO
 | | | ... Result truncated.

Each entry in the student table represents a group of students who follow the same timetable. The parent field is used to maintain a hierachy of groups. A student in group co1.CO.a is also in group co1.CO

- ut_room

id | name | capacity | parent
---|------|---------:|---------
co.117+118 |  | 32 | 
co.117 |  | 16 | co.117+118
co.118 |  | 16 | co.117+118
co.B7 |  | 40 | 
 | | | ... result truncated

All classrooms are listed in the room table. The parent relation is used to maintain groupings. The row co.117+118 represents a pair of rooms - sometimes these rooms are used for a single event.

- ut_event

id | modle | kind | dow | tod | duration | room
---|-------|------|-----|-----|---------:|----------
co12004.L01 | co12004 | L | Wednesday | 11:00 | 1 | cr.SMH
co12004.L02 | co12004 | L | Monday | 17:00 | 1 | cr.B13
co12004.T01 | co12004 | T | Monday | 11:00 | 2 | co.G78+G82
co12004.T02 | co12004 | T | Tuesday | 15:00 | 2 | co.B7
co12004.T03 | co12004 | T | Tuesday | 13:00 | 2 | co.G78+G82
 | | | | | | ... Result truncated.

Every event is associated with a single modle (deliberately mis-spelled version of module - which is a reserved word in one of engines). An event occurs on a particular day of the week (dow) at a particular time of day (tod). Every event has a duration specified as a whole number of hours.

- ut_attends

student | event
--------|------
co1.BAe | co12004.L01
co1.CO | co12004.L01
co1.IS | co12004.L01
co1.SE.pt | co12004.L02
co1.CO.a | co12004.T01
 | ... Result truncated.

The attends table links event to student. It realises the many to many relation.

- ut_teaches

staff | event
------|----------
co.AMn | co12004.L01
co.RK | co12004.L01
co.SRM | co12004.L01
co.RK | co12004.L02
co.SRM | co12004.L02
 | ... Result truncated.

The teaches table links event to staff.

- ut_occurs

event | week
------|-------:
co12004.L01 | 01
co12004.L01 | 02
co12004.L01 | 03
co12004.L01 | 04
co12004.L01 | 05
 | ... Result truncated.

Most events occur on weeks 01 to 13. This table records exactly which weeks are part of the plan for each event.

- ut_modle

id | name
---|---------
co12001 | Rapid Application Development
co12002 | Software Development 1A
co12003 | Professional Skills
co12004 | Applications Workshop
 | ... Result truncated.

By the 21st century, the Gradgrindians had completed their assimilation of Education. The process of "modularisation" was just one of their crimes against learning.

- ut_week

id | wkstart
--:|-------:
01 | 02/10/00
02 | 09/10/00
03 | 16/10/00
 | ...Result truncated


In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app15-1")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@5c779403

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@66031ebd

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val ut_staff = hiveCxt.table("sqlzoo.ut_staff")
val ut_student = hiveCxt.table("sqlzoo.ut_student")
val ut_event = hiveCxt.table("sqlzoo.ut_event")
val ut_room = hiveCxt.table("sqlzoo.ut_room")
val ut_attends = hiveCxt.table("sqlzoo.ut_attends")
val ut_teaches = hiveCxt.table("sqlzoo.ut_teaches")
val ut_occurs = hiveCxt.table("sqlzoo.ut_occurs")
val ut_modle = hiveCxt.table("sqlzoo.ut_modle")
val ut_week = hiveCxt.table("sqlzoo.ut_week")

ut_staff: DataFrame = [id: string, name: string]
ut_student: DataFrame = [id: string, name: string ... 2 more fields]
ut_event: DataFrame = [id: string, modle: string ... 5 more fields]
ut_room: DataFrame = [id: string, name: string ... 2 more fields]
ut_attends: DataFrame = [student: string, event: string]
ut_teaches: DataFrame = [staff: string, event: string]
ut_occurs: DataFrame = [event: string, week: string]
ut_modle: DataFrame = [id: string, name: string]
ut_week: DataFrame = [id: string, wkstart: string]

## 1.
**Give the room id in which the event co42010.L01 takes place.**

In [5]:
(ut_event.filter(ut_event("id")==="co42010.L01")
 .select("room")
 .showHTML())

room
cr.132


## 2.
**For each event in module co72010 show the day, the time and the place.**

In [6]:
(ut_event.filter(ut_event("modle")==="co72010")
 .select("dow", "tod", "room")
 .showHTML())

dow,tod,room
Wednesday,14:00,cr.SMH
Tuesday,09:00,cr.B8
Wednesday,09:00,co.B7
Tuesday,12:00,co.LB42+LB46
Tuesday,11:00,co.G75+G76
Wednesday,16:00,co.LB42+LB46
Thursday,10:00,co.LB42+LB46
Wednesday,13:00,co.117+118


## 3.
**List the names of the staff who teach on module co72010.**

In [7]:
(ut_event.filter(ut_event("modle")==="co72010")
 .join(ut_teaches, (ut_event("id")===ut_teaches("event")))
 .join(ut_staff, (ut_teaches("staff")===ut_staff("id")))
 .select("name")
 .distinct()
 .showHTML())

name
"Cumming, Andrew"
"Chisholm, Ken"


## 4.
**Give a list of the staff and module number associated with events using room cr.132 on Wednesday, include the time each event starts.**

In [8]:
(ut_event.filter((ut_event("room")==="cr.132") && 
                 (ut_event("dow")==="Wednesday"))
 .join(ut_teaches, (ut_event("id")===ut_teaches("event")))
 .join(ut_staff, (ut_teaches("staff")===ut_staff("id")))
 .select("name", "modle")
 .showHTML())

name,modle
"Varey, Alison",co32021
"Murray, Jim",co22009


## 5.
**Give a list of the student groups which take modules with the word 'Database' in the name.**

In [9]:
(ut_student
 .withColumnRenamed("name", "student_group")
 .join(ut_attends, (ut_student("id")===ut_attends("student")))
 .join(ut_event, (ut_attends("event")===ut_event("id")))
 .join(ut_modle.filter(lower(col("name")).like("%database%")), 
       (ut_event("modle")===ut_modle("id")))
 .select("student_group")
 .distinct()
 .showHTML())

student_group
BEng4 Network and Distributing Systems
PgD Information Systems e (OOP)
PgD Information Systems
BSc4 Computing
PgD Software Engineering
null
PgD Software Technology pt. Tues
PgD Information Systems a (HCI)
PgD Information Systems b (DS)
PgD Information Systems d (BT)


In [10]:
spark.stop()